In [5]:
import pandas as pd
import numpy as np
import os

# You will need the Submarket, Market and US Performance trends in a single workbook.

# Set current year of data 
current_yr = 2025

# Paste the REIS export file with sub, msa and national performance trends (ALL)
path = r'C:\Users\Alan Li\Desktop\MMCR\Data Providers\REIS\REIS datahub converter\REIS 4Q24\REIS 4Q24.xlsx'

# Path to save results:
rpath = r'C:\Users\Alan Li\Desktop\MMCR\Data Providers\REIS\REIS datahub converter\REIS 4Q24\Results'

# The two files below reads in lists of markets.

# (For at home - comment out)
msa_82 = pd.read_excel(r'C:\Users\Alan Li\Desktop\MMCR\Data Providers\REIS\REIS datahub converter\REIS formatting script\Market List.xlsx', sheet_name='82 MSA')
TMs = pd.read_excel(r'C:\Users\Alan Li\Desktop\MMCR\Data Providers\REIS\REIS datahub converter\REIS formatting script\Market List.xlsx', sheet_name='TMs')

# # (For in office - comment out)
# msa_82 = pd.read_excel(r'C:\Users\ALi\OneDrive - MMC\Desktop\MMCR\Data Providers\REIS\REIS datahub converter\Market List.xlsx', sheet_name='82 MSA')
# TMs = pd.read_excel(r'C:\Users\ALi\OneDrive - MMC\Desktop\MMCR\Data Providers\REIS\REIS datahub converter\Market List.xlsx', sheet_name='TMs')

sub_id_index = pd.read_excel(r'C:\Users\Alan Li\Desktop\MMCR\Data Providers\REIS\REIS datahub converter\REIS formatting script\REIS submarket ID index_vfinal.xlsx', sheet_name='Sheet1')
TMs_list = TMs['MetroCode'].to_list()
msa_82_list = msa_82['Metro'].to_list()


In [2]:
raw_sub = pd.read_excel(path, sheet_name='Submarket Performance Trends')
sub = pd.DataFrame(raw_sub)

In [3]:
raw_msa = pd.read_excel(path, sheet_name='Market Performance Trends')
msa = pd.DataFrame(raw_msa)

In [4]:
raw_us = pd.read_excel(path, sheet_name='US Performance Trends')
us = pd.DataFrame(raw_us)

In [6]:
# Creates '82 Market' and 'TMs Market' files

msa = msa[msa['Period'].isin(['Y', 'Y-B', 'Q1', 'Q2', 'Q3', 'Q4'])]
msa_df = pd.DataFrame()

msa_df['Metro'] = msa['Market Name']
msa_df['MetroCode'] = msa['Metro Code']
msa_df['Metro.1'] = msa['Market Name']
msa_df['Sector'] = msa['Sector'].replace("Apartment", "Apt")
msa_df['Subsector'] = msa['Building Class'].replace('All', 'ABC')
msa_df['Year'] = msa['Year']
msa_df['Quarter'] = msa['Period'].replace({'Y':5, 'Q1':1, 'Q2':2, 'Q3':3, 'Q4':4, 'Y-B':5})
msa_df['Inventory'] = msa['Inventory (Unit)']
msa_df['Completions'] = msa['Completions']
msa_df['Conversions'] = 0
msa_df['VacPercent'] = msa['Vac %']*100
msa_df['VacantStock'] = msa['Vacant Stock (Unit)']
msa_df['OccupiedStock'] = msa['Occupied Stock (Unit)']
msa_df['NetAbsorption'] = msa['Net Absorption (Unit)']
msa_df['MarketRentDollars'] = msa['Asking Rent/Unit']
msa_df['MarketRentPChange'] = msa['Asking Rent % Chg']*100
msa_df['EffRentDollars'] = round(msa['Effective Rent/Unit'])
msa_df['EffRentPChange'] = msa['Effective Rent % Chg'] * 100
msa_df['ConstrAbsorp'] = msa['Construction/Absorption']
msa_df['AbsorpOccStkP'] = msa['Abs/Occ Stk %']
msa_df['GrRevUnit'] = msa['GrRev/Unit']
msa_df['TotalEmp'] = msa['Total Employment']
msa_df['OfficeEmp'] = msa['Office Employment']
msa_df['IndusEmp'] = msa['Industrial Employment']
msa_df['Households'] = msa['Households']
msa_df['Population'] = msa['Population']
msa_df['HouseholdIncome'] = msa['Household Avg Income']
msa_df['Actual'] = msa['Year'].apply(lambda x: 'A' if x < current_yr else 'F')
msa_df['Tier'] = 1
msa_df['Exp30'] = 'Y'

# Filters

clean_msa_df = msa_df.copy()
clean_msa_df = clean_msa_df.fillna("")
clean_msa_df = clean_msa_df[clean_msa_df['Quarter'].isin(range(6))]
clean_msa_df = clean_msa_df[clean_msa_df['Metro'].isin(msa_82_list)]
clean_msa_df = clean_msa_df[clean_msa_df['Subsector']=="ABC"]
clean_msa_df = clean_msa_df[clean_msa_df['Year'] <= current_yr+4]
clean_msa_df.sort_values(['Metro', 'Year', 'Quarter'], inplace=True)
clean_msa_df.to_excel(os.path.join(rpath, 'REIS_82 Markets_4Q24.xlsx'), index=False)

TMs_msa_df = clean_msa_df[clean_msa_df['MetroCode'].isin(TMs_list)]
TMs_msa_df.to_excel(os.path.join(rpath,'REIS_TMs Markets_4Q24.xlsx'), index=False)




In [7]:
# Creates 'All submarket' and 'TM submarket' tables.

sub_data = sub.copy()
sub_data = sub_data.merge(sub_id_index, left_on=['Market Name', 'Submarket Name'], right_on=['Market Name', 'Subname'])
sub_data = sub_data[sub_data['Period'].isin(['Y', 'Y-B', 'Q1', 'Q2', 'Q3', 'Q4'])]


sub_df = pd.DataFrame()

sub_df['MetroCode'] = sub_data['Metro Code']
sub_df['Metro'] = sub_data['Market Name']
sub_df['SubID'] = sub_data['Subid']
sub_df['Subname'] = sub_data['Submarket Name']
sub_df['Sector'] = sub_data['Sector'].replace("Apartment", "Apt")
sub_df['Subsector'] = sub_data['Building Class'].replace('All', 'ABC')
sub_df['Year'] = sub_data['Year']
sub_df['Quarter'] = sub_data['Period'].replace({'Y':5, 'Q1':1, 'Q2':2, 'Q3':3, 'Q4':4, 'Y-B':5})
sub_df['Inventory'] = sub_data['Inventory (Unit)']
sub_df['Completions'] = sub_data['Completions']
sub_df['Conversions'] = 0
sub_df['VacPercent'] = sub_data['Vac %']*100
sub_df['VacantStock'] = sub_data['Vacant Stock (Unit)']
sub_df['OccupiedStock'] = sub_data['Occupied Stock (Unit)']
sub_df['NetAbsorption'] = sub_data['Net Absorption (Unit)']
sub_df['MarketRentDollars'] = sub_data['Asking Rent/Unit']
sub_df['MarketRentPChange'] = sub_data['Asking Rent % Chg']*100
sub_df['EffRentDollars'] = round(sub_data['Effective Rent/Unit'])
sub_df['EffRentPChange'] = sub_data['Effective Rent % Chg'] * 100
sub_df['ConstrAbsorp'] = sub_data['Construction/Absorption']
sub_df['AbsorpOccStkP'] = sub_data['Abs/Occ Stk %']
sub_df['GrRevUnit'] = sub_data['GrRev/Unit']
sub_df['Actual'] = sub_data['Year'].apply(lambda x: 'A' if x < current_yr else 'F')
sub_df['Tier'] = 1
sub_df['Exp30'] = 'Y'


clean_sub_df = sub_df.copy()
clean_sub_df = clean_sub_df.fillna("")
clean_sub_df = clean_sub_df[clean_sub_df['Quarter'].isin(range(6))]
clean_sub_df = clean_sub_df[clean_sub_df['Metro'].isin(msa_82_list)]
clean_sub_df = clean_sub_df[clean_sub_df['Subsector']=="ABC"]
clean_sub_df = clean_sub_df[clean_sub_df['Year'] <= current_yr+4]
clean_sub_df.sort_values(['Metro', 'SubID', 'Year', 'Quarter'], inplace=True)

clean_sub_df.to_excel(os.path.join(rpath,'REIS_All_Submarkets_4Q24.xlsx'), index=False)

TMs_sub_df = clean_sub_df[clean_sub_df['MetroCode'].isin(TMs_list)]
TMs_sub_df.to_excel(os.path.join(rpath,'REIS_TMs Submarkets_4Q24.xlsx'), index=False)

In [ ]:
# Class cut files for all sets

# Markets by CC
cc_msa_df = msa_df.copy()
cc_msa_df = cc_msa_df.fillna("")
cc_msa_df = cc_msa_df[cc_msa_df['Quarter'].isin(range(6))]
cc_msa_df = cc_msa_df[cc_msa_df['Metro'].isin(msa_82_list)]
cc_msa_df = cc_msa_df[cc_msa_df['Subsector'].isin(['A', 'BC'])]
cc_msa_df = cc_msa_df[cc_msa_df['Year'] <= current_yr+4]
cc_msa_df.sort_values(['Metro', 'Subsector', 'Year', 'Quarter'], inplace=True)
cc_msa_df.to_excel(os.path.join(rpath,'REIS_AllMarkets_cc_4Q24.xlsx'), index=False)

TMs_cc_msa_df = cc_msa_df[cc_msa_df['MetroCode'].isin(TMs_list)]
TMs_cc_msa_df.to_excel(os.path.join(rpath,'REIS_TMs_cc_4Q24.xlsx'), index=False)

# Submarkets by CC
cc_sub_df = sub_df.copy()
cc_sub_df = cc_sub_df.fillna("")
cc_sub_df = cc_sub_df[cc_sub_df['Quarter'].isin(range(6))]
cc_sub_df = cc_sub_df[cc_sub_df['Metro'].isin(msa_82_list)]
cc_sub_df = cc_sub_df[cc_sub_df['Subsector'].isin(['A', 'BC']) ]
cc_sub_df = cc_sub_df[cc_sub_df['Year'] <= current_yr+4]
cc_sub_df.sort_values(['Metro', 'SubID', 'Subsector', 'Year', 'Quarter'], inplace=True)

cc_sub_df.to_excel(os.path.join(rpath,'REIS_AllSubs_cc_4Q24.xlsx'), index=False)

TMs_cc_sub_df = cc_sub_df[cc_sub_df['MetroCode'].isin(TMs_list)]
TMs_cc_sub_df.to_excel(os.path.join(rpath,'REIS_TMsSubs_cc_4Q24.xlsx'), index=False)


In [9]:
usa = us.copy()

usa = usa[usa['Period'].isin(['Y', 'Y-B', 'Q1', 'Q2', 'Q3', 'Q4'])]
usa_df = pd.DataFrame()

usa_df['Metro'] = usa['Country Name'].replace("United States", "US Metro Total")
usa_df['MetroCode'] = "US"
usa_df['Metro.1'] = usa['Country Name'].replace("United States", "US Metro Total")
usa_df['Sector'] = usa['Sector'].replace("Apartment", "Apt")
usa_df['Subsector'] = usa['Building Class'].replace('All', 'ABC')
usa_df['Year'] = usa['Year']
usa_df['Quarter'] = usa['Period'].replace({'Y':5, 'Q1':1, 'Q2':2, 'Q3':3, 'Q4':4, 'Y-B':5})
usa_df['Inventory'] = usa['Inventory (Unit)']
usa_df['Completions'] = usa['Completions']
usa_df['Conversions'] = 0
usa_df['VacPercent'] = usa['Vac %']*100
usa_df['VacantStock'] = usa['Vacant Stock (Unit)']
usa_df['OccupiedStock'] = usa['Occupied Stock (Unit)']
usa_df['NetAbsorption'] = usa['Net Absorption (Unit)']
usa_df['MarketRentDollars'] = usa['Asking Rent/Unit']
usa_df['MarketRentPChange'] = usa['Asking Rent % Chg']*100
usa_df['EffRentDollars'] = round(usa['Effective Rent/Unit'])
usa_df['EffRentPChange'] = usa['Effective Rent % Chg'] * 100
usa_df['ConstrAbsorp'] = usa['Construction/Absorption']
usa_df['AbsorpOccStkP'] = usa['Abs/Occ Stk %']
usa_df['GrRevUnit'] = usa['GrRev/Unit']
usa_df['TotalEmp'] = usa['Total Employment']
usa_df['OfficeEmp'] = usa['Office Employment']
usa_df['IndusEmp'] = usa['Industrial Employment']
usa_df['Households'] = usa['Households']
usa_df['Population'] = usa['Population']
usa_df['HouseholdIncome'] = usa['Household Avg Income']
usa_df['Actual'] = usa['Year'].apply(lambda x: 'A' if x < current_yr else 'F')
usa_df['Tier'] = 3
usa_df['Exp30'] = 'Y'

clean_usa_df = usa_df.copy()
clean_usa_df = clean_usa_df.fillna("")
clean_usa_df = clean_usa_df[clean_usa_df['Quarter'].isin(range(6))]
clean_usa_df = clean_usa_df[clean_usa_df['Year'] <= current_yr+4]
clean_usa_df.sort_values(['Metro', 'Year', 'Quarter'], inplace=True)

# For class ABC (not broken our by class type)
total_usa_df = clean_usa_df[clean_usa_df['Subsector']=="ABC"]
total_usa_df.to_excel(os.path.join(rpath,'REIS_US_Metro_Total_4Q24.xlsx'), index=False)

# For class A and B/C 
cc_usa_df = clean_usa_df[clean_usa_df['Subsector'].isin(['A', 'BC'])]
cc_usa_df2 = cc_usa_df.sort_values(['Subsector', 'Year', 'Quarter'])
cc_usa_df2.to_excel(os.path.join(rpath,'REIS_US_Metro_Total_cc_4Q24.xlsx'), index=False)